In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import rtsvg
rt = rtsvg.RACETrack()

# mostly a duplicate of the circle_routing...

_n_paths_         = 500
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_circle_geoms_    = []
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test, half_sep):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

def svgPathTag(_segments_, stroke='#000000', stroke_width=0.4):
    d = f'M {_segments_[0][0]} {_segments_[0][1]}'
    for i in range(1,len(_segments_)):
        d += f' L {_segments_[i][0]} {_segments_[i][1]}'
    return f'<path d="{d}" stroke="{stroke}" stroke-width="{stroke_width}" fill="none"/>'

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    _max_attempts_ = 100
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), \
                random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
        cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), \
                    random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == _max_attempts_:
        return None
    return cx, cy, r

for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

entries         = [_pts_]
_exit_segments_ = []
_exit_segments_.append(_pts_[2:])
for i in range(_n_paths_):
    _more_pts_ = [_pts_[0], _pts_[1]]
    c          = random.randint(0,len(_circle_geoms_)-1)
    while c == c0:
        c  = random.randint(0,len(_circle_geoms_)-1)
    a = random.random() * 2 * pi
    cx, cy, r = _circle_geoms_[c]
    _more_pts_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
    _more_pts_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))
    _exit_segments_.append(_more_pts_[2:])
    entries.append(_more_pts_)

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Entry Points
svg += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="2" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="2" stroke="#ff0000" fill="#ff0000" />'
svg_base = svg
rt.displaySVG(svg_base + '</svg>')

In [ ]:
xy_tree = rt.xyQuadTree((-10,-10,610,410))

# sort paths -- longest will be first...
exit_sorter = []
for i in range(len(entries)):
    _entry_ = entries[i]
    l = rt.segmentLength((_entry_[0], _entry_[3]))
    exit_sorter.append((l,i))
exit_sorter = sorted(exit_sorter)
exit_sorter.reverse()

# keep track of all of the final paths
paths = []
for i in range(len(entries)):
    paths.append(entries[i])

# plot out the longest path
i_longest        = exit_sorter[0][1]
pts              = entries[i_longest]
_path_           = calculatePathAroundCircles(pts, _circle_geoms_, _radius_inc_test_, _half_sep_)
_path_smooth_    = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))
_pts_total_      = len(_path_smooth_)
_pts_plotted_    = len(_path_smooth_)
paths[i_longest] = _path_smooth_
for i in range(len(_path_smooth_)):
    pt = (_path_smooth_[i][0], _path_smooth_[i][1], i_longest, i)
    xy_tree.add([pt])

# start the svg additions
svg_flat   = svgPathTag(_path_)
svg_smooth = svgPathTag(_path_smooth_)

# analyze the other paths
for i in range(1,len(exit_sorter)):
    i_path         =  exit_sorter[i][1]
    pts            =  entries[i_path]
    _path_         =  calculatePathAroundCircles(pts, _circle_geoms_, _radius_inc_test_, _half_sep_)
    _path_smooth_  =  rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))    
    _pts_total_   +=  len(_path_smooth_)
    _path_merged_  =  [_path_smooth_[-1]]
    for j in range(len(_path_smooth_)-2, 2, -1):
        closest = xy_tree.closest((_path_smooth_[j][0],_path_smooth_[j][1]), n=1)
        _path_merged_.append(_path_smooth_[j])
        if closest[0][0] < 5.0:
            break
    svg_flat      += svgPathTag(_path_)
    svg_smooth    += svgPathTag(_path_merged_)
    _pts_plotted_ += len(_path_merged_)
    for j in range(len(_path_merged_)-3):
        pt = (_path_merged_[j][0], _path_merged_[j][1], i_path, j)
        xy_tree.add([pt])

print(f'Points Total = {_pts_total_} | Points Plotted = {_pts_plotted_}')
rt.tile([svg_base + svg_flat + '</svg>', svg_base + svg_smooth + '</svg>'])

In [ ]:
xy_tree = rt.xyQuadTree((-10,-10,610,410), max_pts_per_node=20)

# keep track of all of the final paths
paths = []
for i in range(len(entries)):
    paths.append(entries[i])

# plot out the longest path
i_longest        = exit_sorter[0][1]
pts              = entries[i_longest]
_path_           = calculatePathAroundCircles(pts, _circle_geoms_, _radius_inc_test_, _half_sep_)
_path_smooth_    = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))
paths[i_longest] = _path_smooth_
for i in range(len(_path_smooth_)):
    pt = (_path_smooth_[i][0], _path_smooth_[i][1], i_longest, i)
    xy_tree.add([pt])

# start the svg additions
svg_smooth    = [svgPathTag(_path_smooth_)]
_pts_plotted_ = len(_path_smooth_)

# analyze the other paths
for i in range(1,len(exit_sorter)):
    i_path        =  exit_sorter[i][1]
    pts           =  entries[i_path]
    _path_        =  calculatePathAroundCircles(pts, _circle_geoms_, _radius_inc_test_, _half_sep_)
    _path_smooth_ =  rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))    
    _path_merged_ =  [_path_smooth_[-1]]
    for j in range(len(_path_smooth_)-2, 2, -1):
        closest = xy_tree.closest((_path_smooth_[j][0],_path_smooth_[j][1]), n=1)
        _path_merged_.append(_path_smooth_[j])
        if closest[0][0] < 5:
            _path_merged_.append((closest[0][1][0], closest[0][1][1]))
            break
    svg_smooth.append(svgPathTag(_path_merged_))
    _pts_plotted_ += len(_path_merged_)
    for j in range(len(_path_merged_)-3):
        pt = (_path_merged_[j][0], _path_merged_[j][1], i_path, j)
        xy_tree.add([pt])
print(f'Points Total = {_pts_total_} | Points Plotted = {_pts_plotted_}')
rt.tile([svg_base + ''.join(svg_smooth) + '</svg>'])

In [ ]:
xy_tree